In [1]:
import subprocess
import sys

def install_if_missing(packages: dict[str, str]):
    """
    Auto-installs missing packages.
    Key = import name, Value = pip install name (if different).
    """
    for import_name, pip_name in packages.items():
        try:
            __import__(import_name)
        except ImportError:
            print(f"  Installing {pip_name}...")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", pip_name, "-q"]
            )
            print(f"  ✓ {pip_name} installed")

install_if_missing({
    "pyarrow":      "pyarrow",
    "pandas":       "pandas",
    "numpy":        "numpy",
    "statsmodels":  "statsmodels",
    "sklearn":      "scikit-learn",
    "matplotlib":   "matplotlib",
})

# ── Imports ───────────────────────────────────────────────────────────────────
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_absolute_error, mean_tweedie_deviance
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

print("✓ All packages ready")

  Installing statsmodels...
  ✓ statsmodels installed
  Installing matplotlib...
  ✓ matplotlib installed
✓ All packages ready


In [2]:
df_freq = pq.read_table("clean_business_claims_freq.parquet").to_pandas()
df_sev = pq.read_table("clean_business_claims_sev.parquet").to_pandas()


In [3]:
df_freq.sort_values(by="solar_system", inplace=True)
df_sev.sort_values(by="solar_system", inplace=True)

In [4]:
#Grouping sev dataset based on policy id and summing claim_amount
agg_dict = {col: 'first' for col in df_sev.columns if col not in ['claim_id', 'claim_seq', 'policy_id', 'claim_amount']}
agg_dict['claim_amount'] = 'sum'

df_grouped_business_claims_sev = (
    df_sev.drop(columns=['claim_id', 'claim_seq'])
    .groupby('policy_id', as_index=False)
    .agg(agg_dict)
)

df_grouped_business_claims_sev.head(10)

,policy_id,station_id,solar_system,production_load,exposure,energy_backup_score,safety_compliance,claim_amount
0,BI-000010,G3,Zeta,0.688,0.300,4,3,4118460.0
1,BI-000015,G2,Helionis Cluster,0.414,0.570,3,5,961887.0
2,BI-000028,A9,Zeta,0.355,0.334,3,1,522526.0
3,BI-000056,B3,Epsilon,0.457,0.761,4,3,1152033.0
4,BI-000062,B2,Epsilon,0.890,0.821,4,4,2061466.0
5,BI-000072,B3,Epsilon,0.023,0.614,2,4,2860153.0
6,BI-000081,G1,Zeta,0.582,0.724,5,4,9022207.0
7,BI-000085,G3,Epsilon,0.671,0.634,1,2,3094557.0
8,BI-000092,G3,Zeta,0.821,0.713,5,3,5026302.0
9,BI-000103,G5,Zeta,0.626,0.628,4,4,2269630.0


In [5]:
#merging the two datasets on policy_id
sev_claim_amounts = df_sev.groupby('policy_id', as_index=False)['claim_amount'].sum()

df_combined = df_freq.merge(
    sev_claim_amounts, 
    on='policy_id', 
    how='left'
)

df_combined['claim_amount'] = df_combined['claim_amount'].fillna(0)

df_combined.columns
zero_ratio = (df_combined['claim_amount'] == 0).mean()
print(f"Zero claims: {zero_ratio:.1%}")

#df_combined.to_csv("bussiness_clams_model_data.csv", index=False)

Zero claims: 92.9%


In [6]:

# ── 0. Imports ────────────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import PoissonRegressor, GammaRegressor, TweedieRegressor
from sklearn.metrics import mean_absolute_error, mean_tweedie_deviance


# ── 1. Schema ─────────────────────────────────────────────────────────────────
NUMERIC_FEATURES = [
    "production_load", "energy_backup_score", "supply_chain_index",
    "avg_crew_exp", "maintenance_freq", "safety_compliance",
]
CATEGORICAL_FEATURES = ["station_id", "solar_system"]
ALL_FEATURES  = NUMERIC_FEATURES + CATEGORICAL_FEATURES
TARGET_FREQ   = "claim_count"
TARGET_SEV    = "claim_amount"
EXPOSURE_COL  = "exposure"


# ── 2. Data Prep ──────────────────────────────────────────────────────────────
def prepare_data(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    df = df.copy()
    df = df[df[EXPOSURE_COL] > 0]
    df = df[df[TARGET_SEV] >= 0]
    df["log_exposure"] = np.log(df[EXPOSURE_COL])

    # Frequency dataset — all rows
    freq_df = df.copy()

    # Severity dataset — only rows WHERE a claim occurred
    sev_df = df[df[TARGET_SEV] > 0].copy()
    # Severity target = average cost per claim
    sev_df["avg_claim_amount"] = sev_df[TARGET_SEV] / sev_df[TARGET_FREQ].clip(lower=1)

    return freq_df, sev_df


# ── 3. Preprocessor ───────────────────────────────────────────────────────────
def build_preprocessor() -> ColumnTransformer:
    return ColumnTransformer([
        ("num", StandardScaler(), NUMERIC_FEATURES),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), CATEGORICAL_FEATURES),
    ])


# ── 4. Stepwise AIC Variable Selection (statsmodels) ─────────────────────────
def backward_aic(
    df: pd.DataFrame,
    feature_cols: list[str],
    target: str,
    family,
    offset_col: str = "log_exposure",
    weight_col: str = None,
) -> list[str]:
    """Backward stepwise selection — drops variables that reduce AIC."""

    remaining = feature_cols.copy()
    current_aic = np.inf

    while True:
        best_aic   = current_aic
        worst_var  = None

        for var in remaining:
            candidate = [v for v in remaining if v != var]
            X = sm.add_constant(df[candidate].astype(float))
            y = df[target].astype(float)

            kwargs = {"offset": df[offset_col].astype(float)} if offset_col else {}
            if weight_col:
                kwargs["freq_weights"] = df[weight_col].astype(float)

            try:
                result = sm.GLM(y, X, family=family, **kwargs).fit(disp=False)
                if result.aic < best_aic:
                    best_aic  = result.aic
                    worst_var = var
            except Exception:
                continue

        if worst_var is None or best_aic >= current_aic:
            break

        print(f"    Dropping '{worst_var}' → AIC {current_aic:.1f} → {best_aic:.1f}")
        remaining.remove(worst_var)
        current_aic = best_aic

    print(f"    ✓ Selected: {remaining}\n")
    return remaining


# ── 5. Interaction Features ───────────────────────────────────────────────────
def add_interactions(df: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    """Add theoretically motivated interaction terms."""
    df = df.copy()
    interactions = {
        "energy_x_safety":    ("energy_backup_score", "safety_compliance"),
        "load_x_maintenance": ("production_load",     "maintenance_freq"),
        "crew_x_supply":      ("avg_crew_exp",        "supply_chain_index"),
    }
    new_cols = []
    for name, (a, b) in interactions.items():
        df[name] = df[a] * df[b]
        new_cols.append(name)
    return df, new_cols


# ── 6. Frequency Model (Poisson GLM) ─────────────────────────────────────────
def fit_frequency_model(
    train_df: pd.DataFrame,
    feature_cols: list[str],
) -> tuple:
    """
    Poisson GLM: E[claim_count] = exp(Xβ + log(exposure))
    Returns (sm_result, selected_features)
    """
    print("── Frequency: Backward AIC selection ──────────────────────────────")
    df_enc = pd.get_dummies(
        train_df[feature_cols + [TARGET_FREQ, "log_exposure"]],
        columns=CATEGORICAL_FEATURES, drop_first=True
    )
    enc_feature_cols = [c for c in df_enc.columns if c not in [TARGET_FREQ, "log_exposure"]]

    selected = backward_aic(
        df_enc, enc_feature_cols, TARGET_FREQ,
        family=sm.families.Poisson(link=sm.families.links.Log()),
        offset_col="log_exposure",
    )

    X = sm.add_constant(df_enc[selected].astype(float))
    model = sm.GLM(
        df_enc[TARGET_FREQ].astype(float), X,
        family=sm.families.Poisson(link=sm.families.links.Log()),
        offset=df_enc["log_exposure"].astype(float),
    ).fit(disp=False)

    return model, selected, df_enc.columns.tolist()


# ── 7. Severity Model (Gamma GLM) ─────────────────────────────────────────────
def fit_severity_model(
    sev_train: pd.DataFrame,
    feature_cols: list[str],
) -> tuple:
    """
    Gamma GLM: E[avg_claim_amount | claim > 0] = exp(Xβ)
    Weighted by claim_count (more claims = more reliable severity estimate).
    """
    print("── Severity: Backward AIC selection ───────────────────────────────")
    df_enc = pd.get_dummies(
        sev_train[feature_cols + ["avg_claim_amount", TARGET_FREQ, "log_exposure"]],
        columns=CATEGORICAL_FEATURES, drop_first=True
    )
    enc_feature_cols = [c for c in df_enc.columns
                        if c not in ["avg_claim_amount", TARGET_FREQ, "log_exposure"]]

    selected = backward_aic(
        df_enc, enc_feature_cols, "avg_claim_amount",
        family=sm.families.Gamma(link=sm.families.links.Log()),
        offset_col=None,  # no offset for severity
        weight_col=TARGET_FREQ,
    )

    X = sm.add_constant(df_enc[selected].astype(float))
    model = sm.GLM(
        df_enc["avg_claim_amount"].astype(float), X,
        family=sm.families.Gamma(link=sm.families.links.Log()),
        freq_weights=df_enc[TARGET_FREQ].astype(float),
    ).fit(disp=False)

    return model, selected, df_enc.columns.tolist()


# ── 8. Prediction ─────────────────────────────────────────────────────────────
def predict_pure_premium(
    df: pd.DataFrame,
    freq_model,
    sev_model,
    freq_selected: list[str],
    sev_selected: list[str],
    all_enc_cols_freq: list[str],
    all_enc_cols_sev: list[str],
) -> np.ndarray:
    """
    Pure Premium = E[frequency] × E[severity]
                 = (predicted claims per unit exposure) × (predicted cost per claim)
    """
    def _encode(df_, cat_cols, all_enc_cols):
        df_enc = pd.get_dummies(df_, columns=cat_cols, drop_first=True)
        # Align columns to training schema
        for col in all_enc_cols:
            if col not in df_enc.columns:
                df_enc[col] = 0
        return df_enc

    # Frequency prediction
    freq_enc = _encode(df[ALL_FEATURES + ["log_exposure"]], CATEGORICAL_FEATURES, all_enc_cols_freq)
    X_freq   = sm.add_constant(freq_enc[freq_selected].astype(float), has_constant="add")
    pred_freq = freq_model.predict(X_freq, offset=freq_enc["log_exposure"].astype(float))

    # Severity prediction
    sev_enc  = _encode(df[ALL_FEATURES], CATEGORICAL_FEATURES, all_enc_cols_sev)
    X_sev    = sm.add_constant(sev_enc[sev_selected].astype(float), has_constant="add")
    pred_sev = sev_model.predict(X_sev)

    # Pure premium = freq rate × severity (already per-unit-exposure from Poisson offset)
    pure_premium = pred_freq * pred_sev
    return pure_premium


# ── 9. Evaluation ─────────────────────────────────────────────────────────────
def evaluate(df: pd.DataFrame, pred_pp: np.ndarray, label: str = ""):
    actual_pp = df[TARGET_SEV] / df[EXPOSURE_COL]
    mae       = mean_absolute_error(actual_pp, pred_pp)
    corr      = np.corrcoef(actual_pp, pred_pp)[0, 1]

    # Lift chart — are high-risk policies ranked correctly?
    n_deciles    = 10
    df_eval      = pd.DataFrame({"actual": actual_pp, "predicted": pred_pp})
    df_eval["decile"] = pd.qcut(df_eval["predicted"], n_deciles, labels=False, duplicates="drop")
    lift         = df_eval.groupby("decile")[["actual", "predicted"]].mean()

    print(f"\n{'═'*45}")
    print(f"  {label}")
    print(f"  MAE (pure premium):    {mae:>15,.2f}")
    print(f"  Pearson correlation:   {corr:>15.4f}")
    print(f"{'═'*45}")

    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(13, 5))

    axes[0].scatter(actual_pp, pred_pp, alpha=0.2, s=8)
    lim = max(actual_pp.max(), pred_pp.max())
    axes[0].plot([0, lim], [0, lim], "r--")
    axes[0].set_xlabel("Actual Pure Premium")
    axes[0].set_ylabel("Predicted Pure Premium")
    axes[0].set_title(f"Actual vs Predicted — {label}")

    axes[1].plot(lift.index, lift["actual"],    "o-", label="Actual")
    axes[1].plot(lift.index, lift["predicted"], "s--", label="Predicted")
    axes[1].set_xlabel("Risk Decile (predicted)")
    axes[1].set_ylabel("Mean Pure Premium")
    axes[1].set_title("Lift Chart by Risk Decile")
    axes[1].legend()

    plt.tight_layout()
    plt.show()

    return {"mae": mae, "correlation": corr, "lift": lift}


# ── 10. Main ──────────────────────────────────────────────────────────────────
def run_freq_sev_model(df: pd.DataFrame):

    # ── Step 1: Prep
    print("── Step 1: Preparing data ──────────────────────────────────────────")
    freq_df, sev_df = prepare_data(df)

    freq_df, interaction_cols = add_interactions(freq_df)
    sev_df,  _               = add_interactions(sev_df)
    extended_features = ALL_FEATURES + interaction_cols

    zero_pct = (freq_df[TARGET_SEV] == 0).mean()
    print(f"  Frequency rows : {len(freq_df):,}")
    print(f"  Severity rows  : {len(sev_df):,}  ({zero_pct:.1%} zero-claim policies excluded)")

    # ── Step 2: Split
    print("\n── Step 2: Train / Test split ──────────────────────────────────────")
    freq_df["_zero"] = (freq_df[TARGET_SEV] == 0).astype(int)
    train_freq, test_freq = train_test_split(
        freq_df, test_size=0.2, random_state=42, stratify=freq_df["_zero"]
    )
    train_freq = train_freq.drop(columns="_zero")
    test_freq  = test_freq.drop(columns="_zero")

    train_sev  = sev_df[sev_df.index.isin(train_freq.index)]
    test_sev   = sev_df[sev_df.index.isin(test_freq.index)]
    print(f"  Train freq: {len(train_freq):,}  |  Train sev: {len(train_sev):,}")
    print(f"  Test  freq: {len(test_freq):,}  |  Test  sev: {len(test_sev):,}")

    # ── Step 3: Frequency model
    print("\n── Step 3: Frequency Model (Poisson GLM) ───────────────────────────")
    freq_model, freq_selected, freq_enc_cols = fit_frequency_model(train_freq, extended_features)
    print(freq_model.summary())

    # ── Step 4: Severity model
    print("\n── Step 4: Severity Model (Gamma GLM) ──────────────────────────────")
    sev_model, sev_selected, sev_enc_cols = fit_severity_model(train_sev, extended_features)
    print(sev_model.summary())

    # ── Step 5: Predict pure premium
    print("\n── Step 5: Pure Premium = Frequency × Severity ─────────────────────")
    train_pp = predict_pure_premium(
        train_freq, freq_model, sev_model,
        freq_selected, sev_selected, freq_enc_cols, sev_enc_cols
    )
    test_pp  = predict_pure_premium(
        test_freq, freq_model, sev_model,
        freq_selected, sev_selected, freq_enc_cols, sev_enc_cols
    )

    # ── Step 6: Evaluate
    print("\n── Step 6: Evaluation ──────────────────────────────────────────────")
    train_metrics = evaluate(train_freq, train_pp, label="Train")
    test_metrics  = evaluate(test_freq,  test_pp,  label="Test")

    return {
        "freq_model":    freq_model,
        "sev_model":     sev_model,
        "freq_selected": freq_selected,
        "sev_selected":  sev_selected,
        "freq_enc_cols": freq_enc_cols,
        "sev_enc_cols":  sev_enc_cols,
        "train_metrics": train_metrics,
        "test_metrics":  test_metrics,
    }


# ── Run ───────────────────────────────────────────────────────────────────────
results = run_freq_sev_model(df_combined)

── Step 1: Preparing data ──────────────────────────────────────────
  Frequency rows : 99,923
  Severity rows  : 7,055  (92.9% zero-claim policies excluded)

── Step 2: Train / Test split ──────────────────────────────────────
  Train freq: 79,938  |  Train sev: 5,644
  Test  freq: 19,985  |  Test  sev: 1,411

── Step 3: Frequency Model (Poisson GLM) ───────────────────────────
── Frequency: Backward AIC selection ──────────────────────────────
    Dropping 'station_id_A6' → AIC inf → 57923.8


KeyboardInterrupt: 